# Ferring - List
## Next time
* You may update the column positions. On Mac, you do this:
 * Right click on PDF in finder
 * Open with -> Preview (Vorschau)
 * Tools (Werkzeuge) -> Show information (Informationen einblenden)
 * Rechteckige Auswahl

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [10]:
# HCP
columns = [92.8, 194.03, 254.53, 393.35, 424.59, 466.23, 514.31, 565.69, 613.29, 662.05, 719.59]
df_hcp = tabula.read_pdf("hcp_2018_offenlegung.pdf", pages='all', lattice=False, columns=columns, guess=False, pandas_options={'header': None})
df_hcp['type'] = 'hcp'

#HCO
columns = [14.41, 227.18, 300.16, 426.84, 482.12, 533.73, 576.96, 631.57, 680.82, 732.14, 794.54]
df_hco = tabula.read_pdf("hco_2018_offenlegung.pdf", pages='all', lattice=False, columns=columns, guess=False, pandas_options={'header': None})
df_hco['type'] = 'hco'

df = pd.concat([df_hcp, df_hco], sort=False)

## Format Table

In [16]:
df_export = df.copy()

#Rename
df_export.columns = ['empty0', 'name', 'location', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total', 'type']

#Remove empty rows
df_export = df_export.dropna(subset=['location'], how='all')

#Remove by String Comparision
df_export = df_export[~df_export.name.str.contains('Name')]
df_export = df_export[~df_export.name.str.contains('Number of recipients')]

#Remove empty columns
df_export.drop(columns=['empty0', 'empty1'], inplace=True)

#Remove Title
df_export['name'] = df_export['name'].str.replace(regex_title, '', regex=True, case=False)

#Add missing fields
df_export = add_plz(df_export)
df_export = add_uci(df_export)
df_export['country'] = 'CH'

#Numberize
df_export = cleanup_number(df_export)
df_export = replace_in_number(df_export, '-', '')
df_export = replace_in_number(df_export, '’', '')
df_export = amounts_to_number(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'ferring')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)